<a href="https://colab.research.google.com/github/pikey-msc/RiesgosFinancieros/blob/master/2022-2/Insumos/RH_acciones_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Riesgo de Mercado (Simulación Histórica)

Existen trés métodos principales para medir el riesgo de mercado
* Simulación histórica
* Paramétrico
* Simulación de Monte Carlo



   **Pérdidas y Ganancias simulación histórica (sin alisado)**
   
Un enfoque más claro es suponer que tenemos:
1.   Una matriz $X_{(n+1)\times m}$ de $m$ factores de riesgo y $n+1$ observaciones. 
2.   Denotemos el vector de precios actual como $X_{00}:=(x_{0,1},x_{0,2},\ldots,x_{0,m})$. 
3.   Sea $r$ el número de instrumentos de un portafolio, entonces cada instrumento tiene una función de valuación $f_i\colon A_i \to R$ para todo $x\in X$, $i=1,\ldots, r$, donde $A_i\subset X_i$ con $\#(A_i)\leq \#(X_i)$.
4.   Sea $r$ el número de instrumentos de un portafolio, entonces cada instrumento tiene una función de valuación $f_i\colon A_i \to R$ para todo $x\in X$, $i=1,\ldots, r$, donde $A_i\subset X_i$ con $\#(A_i)\leq \#(X_i)$.
5.   Sea $M_{1\times r}=(m_1,\ldots,m_r)$ el vector de posiciones nominales de cada instrumento, es decir, el número de contratos que se tienen por instrumento $m_i\in R$ $(i=1,\ldots,r)$.

In [ ]:
!pip install yfinance
import yfinance as yf
import pandas as pd


In [ ]:
start = '2020-01-01'

symbol = ['AAPL','MSFT','SQ','AMZN']
stock_data = yf.download(symbol,start)
# stock_data = stock_data.history(start=start)#, end=end) 
# symbol1='APPL'
# stock_data = yf.Ticker(symbol)
# stock_data = stock_data.history(start=start)#, end=end) 


[*********************100%***********************]  4 of 4 completed



##Paso 1.
 (Generación de rendimientos) Construir ${\Delta X}_{n\times m}$ que es la matriz de diferencias basados en el operador $T_j$, es decir
		$${\Delta X}_t=\Big[ T_j(\frac{x_{t,1}}{x_{t+1,1}}), T_j(\frac{x_{t,2}}{x_{t+1,2}}),\ldots, T_j(\frac{x_{t,m}}{x_{t+1,m}})\Big]\quad t=0,1,\ldots, n-1.$$

In [ ]:
# yf.download?
print(stock_data.Close.head(5))

import numpy as np
X=stock_data.Close
X0=stock_data.Close.tail(1)
DeltaX=np.log(stock_data.Close/stock_data.Close.shift(1))

print(DeltaX.head())
print(X0)


                 AAPL         AMZN        MSFT         SQ
Date                                                     
2020-01-02  75.087502  1898.010010  160.619995  63.830002
2020-01-03  74.357498  1874.969971  158.619995  63.000000
2020-01-06  74.949997  1902.880005  159.029999  62.570000
2020-01-07  74.597504  1906.859985  157.580002  64.589996
2020-01-08  75.797501  1891.969971  160.089996  67.599998
                AAPL      AMZN      MSFT        SQ
Date                                              
2020-01-02       NaN       NaN       NaN       NaN
2020-01-03 -0.009770 -0.012213 -0.012530 -0.013089
2020-01-06  0.007937  0.014776  0.002581 -0.006849
2020-01-07 -0.004714  0.002089 -0.009160  0.031774
2020-01-08  0.015958 -0.007839  0.015803  0.045548
                  AAPL         AMZN        MSFT          SQ
Date                                                       
2022-03-08  157.440002  2720.290039  275.850006  100.620003


In [ ]:
import pandas as pd
M=np.array([1000,5000,2000,100])
print(M)

#Valoración

Y0=X0.squeeze()*M

Y0



[1000 5000 2000  100]


AAPL    1.574400e+05
AMZN    1.360145e+07
MSFT    5.517000e+05
SQ      1.006200e+04
Name: 2022-03-08 00:00:00, dtype: float64

##Paso 2. 
Construir ${X^\ast}_{n\times m}$ que es la matriz de factores de riesgo simulada basada en el vector de precios actual $X_{00}$
		$$X_{t}^\ast=\Big[x_{0,1}T^{-1}_j(\Delta x_{t,1}), x_{0,2}T^{-1}_j(\Delta x_{t,2}),\ldots, x_{0,m}T^{-1}_j(\Delta x_{t,m})\Big]\quad (t=1,\ldots,n\quad j=1,2),$$
		donde $T_1(x)=\ln(x)$, $T_2(x)=x-1$.



In [ ]:
X_s=np.exp(DeltaX.squeeze())*X0.squeeze()
X_s.head

<bound method NDFrame.head of                   AAPL         AMZN        MSFT          SQ
Date                                                       
2020-01-02         NaN          NaN         NaN         NaN
2020-01-03  155.909365  2687.268301  272.415191   99.311609
2020-01-06  158.694523  2760.783161  276.563028   99.933231
2020-01-07  156.699555  2725.979679  273.334873  103.868398
2020-01-08  159.972628  2699.048229  280.243850  105.309064
...                ...          ...         ...         ...
2022-03-02  160.681415  2736.677401  280.750672   99.183262
2022-03-03  157.128070  2645.972976  271.926236   92.490074
2022-03-04  154.541813  2678.768039  270.200984   93.836826
2022-03-07  153.705909  2567.354151  265.429277   92.109059
2022-03-08  155.601719  2691.821109  272.823581  103.829194

[550 rows x 4 columns]>

## Paso 3. 
Construcción de la matriz de reevaluación del portafolio basada en los escenarios históricos $Y_{n\times r}$ de todos los instrumentos financieros, es decir
		$$Y_t=[m_1f_1(X_{t}^\ast), m_2f_2(X_{t}^\ast),\ldots, m_rf_r(X_{t}^\ast)]\quad (t=1,\ldots,n).$$
    


In [ ]:
Y_s=X_s*M.squeeze()
Y_s.head


<bound method NDFrame.head of                      AAPL          AMZN           MSFT            SQ
Date                                                                
2020-01-02            NaN           NaN            NaN           NaN
2020-01-03  155909.365145  1.343634e+07  544830.381663   9931.160882
2020-01-06  158694.522988  1.380392e+07  553126.055785   9993.323081
2020-01-07  156699.554862  1.362990e+07  546669.745338  10386.839764
2020-01-08  159972.627706  1.349524e+07  560487.700898  10530.906360
...                   ...           ...            ...           ...
2022-03-02  160681.414906  1.368339e+07  561501.343133   9918.326193
2022-03-03  157128.069865  1.322986e+07  543852.472410   9249.007435
2022-03-04  154541.812972  1.339384e+07  540401.967425   9383.682578
2022-03-07  153705.908873  1.283677e+07  530858.553099   9210.905926
2022-03-08  155601.719359  1.345911e+07  545647.161222  10382.919425

[550 rows x 4 columns]>

## Paso 4. 
Construcción de la matriz de pérdidas y ganancias del portafolio basada en los escenarios históricos $\Delta Y_{n\times r}$ de todos los instrumentos financieros, es decir
		$$\Delta Y_t=Y_t-Y_{0}=[m_1(f_1(X_{00})-f_1(X_{t}^\ast)), m_2( f_2(X_{00})-f_2(X_{t}^\ast)),\ldots, m_r(f_r(X_{00})-f_r(X_{t}^\ast))],$$
		para $(t=1,\ldots,n)$. Se puede obtener el vector de pérdidas totales ${\Delta YT}_{n\times m}$ muy fácilmente, esto es $\Delta YT=\sum_{k=1}^r Y_{kt}$ $(t=1,\ldots,n)$, incluso se puede hacer lo mismo por tipo de riesgo.


In [ ]:
PL=Y_s-Y0

cols=PL.shape[1]

PL['Total']=PL.sum(axis=1)
PL.head

<bound method NDFrame.head of                    AAPL           AMZN          MSFT          SQ          Total
Date                                                                           
2020-01-02          NaN            NaN           NaN         NaN       0.000000
2020-01-03 -1530.637296 -165108.688676  -6869.630544 -130.839392 -173639.795908
2020-01-06  1254.520547  202465.610530   1426.043578  -68.677193  205077.497462
2020-01-07  -740.447580   28448.197461  -5030.266869  324.839490   23002.322502
2020-01-08  2532.625265 -106209.052686   8787.688691  468.906085  -94419.832644
...                 ...            ...           ...         ...            ...
2022-03-02  3241.412464   81936.810929   9801.330926 -143.674082   94835.880237
2022-03-03  -311.932576 -371585.316485  -7847.539797 -812.992839 -380557.781698
2022-03-04 -2898.189469 -207610.000823 -11298.044782 -678.317697 -222484.552771
2022-03-07 -3734.093568 -764679.439354 -20841.459108 -851.094349 -790106.086379
2022-03-08

##Paso 5. 
Obtenerla medida de riesgo basado en un nivel de confianza de la matriz ${\Delta Y}_{\cdot k}$ $(k=1,\ldots,r)$ y $\Delta YT$.

Las pérdidas y ganancias vía simulación histórica sin alisado es donde se ponderan los escenarios históricos con un mismo peso para todos, es decir cada escenario tiene una probabilidad de ocurrencia de $1/n$.

In [ ]:
alpha = 0.98 #@param {type:"slider", min:0.8, max:1, step:0.001}

###Valor en Riesgo (VaR)

Es un cuantil a un $\alpha$ de confianza de la distribución de pérdidas y ganancias, generalmente se expresa en términos monetarios.

#VaR

El valor en riesgo (VaR) es una estadística utilizada para tratar de cuantificar el nivel de riesgo financiero dentro de una empresa o cartera durante un período de tiempo específico. VaR proporciona una estimación de la pérdida máxima de una posición o cartera determinada durante un período de tiempo, y puede calcularla a través de varios niveles de confianza.

Estimar el riesgo de una cartera es importante para el crecimiento del capital a largo plazo y la gestión del riesgo, en particular dentro de empresas o instituciones más grandes. El VaR generalmente se enmarca como algo como esto:

“Tenemos un VaR de cartera de 250.000 USD para el próximo mes al 95% de confianza”
Esto significa que, con un 95% de confianza, podemos decir que la pérdida de la cartera no superará los 250.000 USD en un mes.
En esta publicación, lo guiaré a través de los pasos para calcular esta métrica en una cartera de acciones.

In [ ]:
VaR=PL.quantile(1-alpha, numeric_only=False)

print(VaR)

print(sum(Y0))

AAPL      -7298.874401
AMZN    -630538.408388
MSFT     -23795.190363
SQ         -813.530388
Total   -677462.575261
Name: 0.020000000000000018, dtype: float64
14320652.210235596


###Valor en Riesgo Condicional o "Expected Shortfall"

Es el promedio de las pérdidas que son más severas que el Valor en Riesgo ($VaR_\alpha$) a un $\alpha$ de confianza, es decir:
$$E(x|x\le VaR_\alpha)$$


In [ ]:
def VaR_Cond(x , VaR ):
  return np.mean(x[x <= VaR])

In [ ]:

VaR_C=[]

for i in range(0,PL.shape[1]):
  VC=VaR_Cond(PL.iloc[:,i],VaR[i])
  VaR_C.append(VC)


In [ ]:

print(VaR)
print(VaR_C)

pd.concat([VaR,pd.DataFrame(VaR_C).squeeze()], axis=1, ignore_index=True)

AAPL      -7298.874401
AMZN    -630538.408388
MSFT     -23795.190363
SQ         -813.530388
Total   -677462.575261
Name: 0.020000000000000018, dtype: float64
[-11261.958141897987, -846332.0609951812, -36937.994447413366, -1200.8464660778375, -881792.5615757715]


,0,1
AAPL,-7298.874401,NaN
AMZN,-630538.408388,NaN
MSFT,-23795.190363,NaN
SQ,-813.530388,NaN
Total,-677462.575261,NaN
0,NaN,-11261.958142
1,NaN,-846332.060995
2,NaN,-36937.994447
3,NaN,-1200.846466
4,NaN,-881792.561576
